In [1]:
# Import the dependencies.
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from citipy import citipy
import requests
from config import weather_api_key
from datetime import datetime

In [2]:
# use api call to create weather url
url = "http://api.openweathermap.org/data/2.5/weather?units=Imperial&APPID=" + weather_api_key

In [3]:
# Create a set of random latitude and longitude combinations.
lats = np.random.uniform(low=-90.000, high=90.000, size=1500)
lngs = np.random.uniform(low=-180.000, high=180.000, size=1500)
lat_lngs = zip(lats, lngs)
lat_lngs

In [4]:
coordinates = list(lat_lngs)

In [5]:
# Create a list for holding the cities.
cities = []
# Identify the nearest city for each latitude and longitude combination.
for coordinate in coordinates:
    city = citipy.nearest_city(coordinate[0], coordinate[1]).city_name

    # If the city is unique, then we will add it to the cities list.
    if city not in cities:
        cities.append(city)
# Print the city count to confirm sufficient count.
len(cities)

612

In [6]:
# Create an empty list to hold the weather data.
city_data = []
# Print the beginning of the logging.
print("Beginning Data Retrieval     ")
print("-----------------------------")

# Create counters.
record_count = 1
set_count = 1
# Loop through all the cities in the list.
for i, city in enumerate(cities):

    # Group cities in sets of 50 for logging purposes.
    if (i % 50 == 0 and i >= 50):
        set_count += 1
        record_count = 1
    # Create endpoint URL with each city.
    city_url = url + "&q=" + city.replace(" ","+")

    # Log the URL, record, and set numbers and the city.
    print(f"Processing Record {record_count} of Set {set_count} | {city}")
    # Add 1 to the record count.
    record_count += 1
    # Run an API request for each of the cities.
    try:
        # Parse the JSON and retrieve data.
        city_weather = requests.get(city_url).json()
        # Parse out the needed data.
        city_lat = city_weather["coord"]["lat"]
        city_lng = city_weather["coord"]["lon"]
        city_max_temp = city_weather["main"]["temp_max"]
        city_humidity = city_weather["main"]["humidity"]
        city_clouds = city_weather["clouds"]["all"]
        city_wind = city_weather["wind"]["speed"]
        city_country = city_weather["sys"]["country"]
        # Convert the date to ISO standard.
        city_date = datetime.utcfromtimestamp(city_weather["dt"]).strftime('%Y-%m-%d %H:%M:%S')
        # Append the city information into city_data list.
        city_data.append({"City": city.title(),
                          "Lat": city_lat,
                          "Lng": city_lng,
                          "Max Temp": city_max_temp,
                          "Humidity": city_humidity,
                          "Cloudiness": city_clouds,
                          "Wind Speed": city_wind,
                          "Country": city_country,
                          "Date": city_date})

# If an error is experienced, skip the city.
    except:
        print("City not found. Skipping...")
        pass

# Indicate that Data Loading is complete.
print("-----------------------------")
print("Data Retrieval Complete      ")
print("-----------------------------")

Beginning Data Retrieval     
-----------------------------
Processing Record 1 of Set 1 | cidreira
Processing Record 2 of Set 1 | lebu
Processing Record 3 of Set 1 | mataura
Processing Record 4 of Set 1 | bengkulu
Processing Record 5 of Set 1 | illoqqortoormiut
City not found. Skipping...
Processing Record 6 of Set 1 | lavrentiya
Processing Record 7 of Set 1 | bluff
Processing Record 8 of Set 1 | atuona
Processing Record 9 of Set 1 | saint-augustin
Processing Record 10 of Set 1 | opuwo
Processing Record 11 of Set 1 | molteno
Processing Record 12 of Set 1 | hithadhoo
Processing Record 13 of Set 1 | jamestown
Processing Record 14 of Set 1 | saskylakh
Processing Record 15 of Set 1 | portland
Processing Record 16 of Set 1 | rikitea
Processing Record 17 of Set 1 | horsham
Processing Record 18 of Set 1 | malakal
Processing Record 19 of Set 1 | albany
Processing Record 20 of Set 1 | ushuaia
Processing Record 21 of Set 1 | bredasdorp
Processing Record 22 of Set 1 | jeremie
Processing Record 2

Processing Record 35 of Set 4 | aflu
City not found. Skipping...
Processing Record 36 of Set 4 | ponta do sol
Processing Record 37 of Set 4 | fairbanks
Processing Record 38 of Set 4 | port blair
Processing Record 39 of Set 4 | solovetskiy
City not found. Skipping...
Processing Record 40 of Set 4 | kurilsk
Processing Record 41 of Set 4 | yanam
Processing Record 42 of Set 4 | amderma
City not found. Skipping...
Processing Record 43 of Set 4 | palmer
Processing Record 44 of Set 4 | kloulklubed
Processing Record 45 of Set 4 | iqaluit
Processing Record 46 of Set 4 | inongo
Processing Record 47 of Set 4 | coquimbo
Processing Record 48 of Set 4 | les cayes
Processing Record 49 of Set 4 | cherskiy
Processing Record 50 of Set 4 | jinxiang
Processing Record 1 of Set 5 | lucapa
Processing Record 2 of Set 5 | san patricio
Processing Record 3 of Set 5 | yellowknife
Processing Record 4 of Set 5 | ancud
Processing Record 5 of Set 5 | east millcreek
Processing Record 6 of Set 5 | tiarei
Processing Rec

Processing Record 27 of Set 8 | kamyshin
Processing Record 28 of Set 8 | moron
Processing Record 29 of Set 8 | tocache
Processing Record 30 of Set 8 | emerald
Processing Record 31 of Set 8 | lakes entrance
Processing Record 32 of Set 8 | yashan
Processing Record 33 of Set 8 | inuvik
Processing Record 34 of Set 8 | san vicente
Processing Record 35 of Set 8 | zeerust
Processing Record 36 of Set 8 | pincher creek
Processing Record 37 of Set 8 | muros
Processing Record 38 of Set 8 | dryden
Processing Record 39 of Set 8 | roma
Processing Record 40 of Set 8 | burnie
Processing Record 41 of Set 8 | garh maharaja
Processing Record 42 of Set 8 | pisco
Processing Record 43 of Set 8 | minab
Processing Record 44 of Set 8 | nosy varika
Processing Record 45 of Set 8 | dukat
Processing Record 46 of Set 8 | san matias
Processing Record 47 of Set 8 | paka
Processing Record 48 of Set 8 | lunglei
Processing Record 49 of Set 8 | san quintin
Processing Record 50 of Set 8 | clyde river
Processing Record 1 o

Processing Record 15 of Set 12 | dayong
Processing Record 16 of Set 12 | anori
Processing Record 17 of Set 12 | utiroa
City not found. Skipping...
Processing Record 18 of Set 12 | puerto plata
Processing Record 19 of Set 12 | evensk
Processing Record 20 of Set 12 | los llanos de aridane
Processing Record 21 of Set 12 | gloversville
Processing Record 22 of Set 12 | whitehorse
Processing Record 23 of Set 12 | yerbogachen
Processing Record 24 of Set 12 | straumen
Processing Record 25 of Set 12 | half moon bay
Processing Record 26 of Set 12 | canto do buriti
Processing Record 27 of Set 12 | yaan
Processing Record 28 of Set 12 | lastoursville
Processing Record 29 of Set 12 | ilulissat
Processing Record 30 of Set 12 | solsvik
City not found. Skipping...
Processing Record 31 of Set 12 | umzimvubu
City not found. Skipping...
Processing Record 32 of Set 12 | kaeo
Processing Record 33 of Set 12 | alyangula
Processing Record 34 of Set 12 | terenga
City not found. Skipping...
Processing Record 35 

In [18]:
#create city data dataframe
city_data_df = pd.DataFrame(city_data)
city_data_df.head(10)
new_column_order = ["City", "Country", "Date", "Lat", "Lng", "Max Temp", "Humidity", "Cloudiness", "Wind Speed"]
city_data_df = city_data_df[new_column_order]

city_data_df.head(10)

,City,Country,Date,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed
0,Cidreira,BR,2021-07-12 15:36:51,-30.1811,-50.2056,66.88,65,35,9.69
1,Lebu,CL,2021-07-12 15:36:58,-37.6167,-73.6500,51.04,63,0,5.14
2,Mataura,NZ,2021-07-12 15:35:58,-46.1927,168.8643,39.79,98,100,1.90
3,Bengkulu,ID,2021-07-12 15:36:12,-3.8004,102.2655,74.64,91,66,4.94
4,Lavrentiya,RU,2021-07-12 15:36:06,65.5833,-171.0000,44.65,91,31,5.75
5,Bluff,NZ,2021-07-12 15:36:41,-46.6000,168.3333,46.09,75,96,5.39
6,Atuona,PF,2021-07-12 15:36:21,-9.8000,-139.0333,78.75,68,34,19.01
7,Saint-Augustin,CA,2021-07-12 15:36:47,51.2260,-58.6502,56.61,78,100,10.51
8,Opuwo,NA,2021-07-12 15:37:48,-18.0607,13.8400,76.32,18,17,6.22
9,Molteno,ZA,2021-07-12 15:37:48,-31.3968,26.3625,58.17,28,52,34.85


In [20]:
# Create the output file (CSV).
output_data_file = "weather_data/cities.csv"
# Export the City_Data into a CSV.
city_data_df.to_csv(output_data_file, index_label="City_ID")